# Gearing Calculator
(C)2019 John Lam. Released under the MIT license

## Spikes

- ~~Define a simple calculator that computes a gearing table using a cassette and crank configuration.~~
- ~~Create a different gearing table that computes the % gap between the previous to the next~~
- ~~Colorize the gearing table so that you see <5% jumps as green and 5-10% jumps as yellow and >10% jumps as red~~
- Create a database of gearing tables in some kind of CSV format that is read into a pandas dataframe 
- Use some kind of simple widget to do computations with drop downs against the gearing combinations


## Fixed configuration cassette and crank

Given `cassette` and `crank` lists that hold the gearing ratios, compute the gear inches.

In [1]:
cassette = [11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 28]
crankset = [34, 50]

In [5]:
import numpy as np
import pandas as pd

In [13]:
data = []
for gear in cassette:
    data.append(
        np.array(
            [crank_gear/gear * 27 for crank_gear in crankset]
        )
    )

df = pd.DataFrame(data, index=cassette, columns=crankset)
df.round(1)

,34,50
11,83.5,122.7
12,76.5,112.5
13,70.6,103.8
14,65.6,96.4
15,61.2,90.0
17,54.0,79.4
19,48.3,71.1
21,43.7,64.3
23,39.9,58.7
25,36.7,54.0


## Compute % jumps between gears

Given a cassette and a crankset, compute jumps going from smaller to larger gears as percentages.

In [20]:
df = pd.DataFrame(index = cassette)

percentage = [0]
for crank_gear in crankset:
    for i in range(len(cassette) - 1):
        percentage.append((cassette[i + 1] - cassette[i])/crank_gear * 100)
    df[crank_gear] = percentage
    percentage = [0]
    
df.round(1)

,34,50
11,0.0,0.0
12,2.9,2.0
13,2.9,2.0
14,2.9,2.0
15,2.9,2.0
17,5.9,4.0
19,5.9,4.0
21,5.9,4.0
23,5.9,4.0
25,5.9,4.0


## Conditional formatting

Colorize the percentage output:
- Green is <5%
- Yellow is 5-10%
- Red is 10%+

In [25]:
def colorize(val):
    if val < 5:
        color = 'green'
        text = 'white'
    elif val >= 5 and val < 10:
        color = 'yellow'
        text = 'black'
    else:
        color = 'red'
        text = 'white'
    
    return f"background-color: {color}; color: {text}"

In [28]:
df.round(1).style.applymap(colorize)

,34,50
11,0,0
12,2.9,2
13,2.9,2
14,2.9,2
15,2.9,2
17,5.9,4
19,5.9,4
21,5.9,4
23,5.9,4
25,5.9,4


## Database of cassettes and cranksets

The simplest thing to do is to create these things as dicts. In the future, we will change this thing to use some kind of file-based database (CSV?)

In [33]:
cassettes = {
    'Shimano 11-46T': [11, 13, 15, 17, 19, 21, 24, 28, 32, 37, 46],
    'Shimano 11-42T': [11, 13, 15, 17, 19, 21, 24, 28, 32, 37, 42],
    'Shimano 11-40T': [11, 13, 15, 17, 19, 21, 24, 27, 31, 35, 40],
    'Shimano 11-32T': [11, 12, 13, 14, 16, 18, 20, 22, 25, 28, 32],
    'Shimano 11-28T': [11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 28],
    'Shimano 11-25T': [11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 25],
    'Shimano 11-23T': [11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23],
    'Shimano 12-25T': [12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 25],
    'Shimano 12-28T': [12, 13, 14, 15, 16, 17, 19, 21, 23, 25, 28],
}

cranksets = {
    'Compact': [50, 34],
    'Cross': [46, 36],
}

In [31]:
def compute_percentages(crankset, cassette):
    df = pd.DataFrame(index = cassette)

    percentage = [0]
    for crank_gear in crankset:
        for i in range(len(cassette) - 1):
            percentage.append((cassette[i + 1] - cassette[i])/crank_gear * 100)
        df[crank_gear] = percentage
        percentage = [0]
        
    def colorize(val):
        if val < 5:
            color = 'green'
            text = 'white'
        elif val >= 5 and val < 10:
            color = 'yellow'
            text = 'black'
        else:
            color = 'red'
            text = 'white'

        return f"background-color: {color}; color: {text}"
    
    return df.round(1).style.applymap(colorize)

In [45]:
gravel_wheel = compute_percentages(cranksets['Compact'], cassettes['Shimano 11-46T'])
gravel_wheel.name = "Gravel Wheel"
road_wheel = compute_percentages(cranksets['Compact'], cassettes['Shimano 11-32T'])

OMG what a hack.

In [46]:
from IPython.display import display, HTML

display(gravel_wheel)
display(road_wheel)

HTML('<style>.output {flex-direction: row;}</style>')

,50,34
11,0,0
13,4,5.9
15,4,5.9
17,4,5.9
19,4,5.9
21,4,5.9
24,6,8.8
28,8,11.8
32,8,11.8
37,10,14.7


,50,34
11,0,0
12,2,2.9
13,2,2.9
14,2,2.9
16,4,5.9
18,4,5.9
20,4,5.9
22,4,5.9
25,6,8.8
28,6,8.8


In [49]:
def compute_gear_inches(cassette, crankset):
    df = pd.DataFrame(index=cassette)
    
    data = []
    for crank_gear in crankset:
        for gear in cassette:
            data.append(gear / crank_gear * 27)
        df[crank_gear] = data
        data = []
    return df

In [ ]:
gravel_gearinches = compute_gear_inches(cassettes['Shimano 11-46T'])